In [2]:
#Required libraries
from rdkit.Chem import AllChem
from rdkit import Chem
from ase.visualize import view
from ase import Atoms
from ase import io
from ase.optimize import BFGS
from xtb.ase.calculator import XTB
from ase.constraints import FixInternals
from ase.mep import NEB
from ase.optimize import FIRE
from ase.calculators.dftb import Dftb
import os
import shutil
from ase.io import read, write
import glob
import pandas as pd
import matplotlib.pyplot as plt


/var/folders/9n/j1gq899160g4whl8ht4097300000gn/T/ipykernel_21765/852910052.py:5: UserWarning: pexpect.screen and pexpect.ANSI are deprecated. We recommend using pyte to emulate a terminal screen: https://pypi.python.org/pypi/pyte
  from pexpect.screen import constrain


In [ ]:
# Molecule preparation
protonated_mol = Chem.MolFromSmiles('C[NH3+]')      # Smiles of the molecule
protonated_mol = Chem.AddHs(protonated_mol)         # Adding Hydrogen
AllChem.EmbedMolecule(protonated_mol)               # Making a 3D structure
AllChem.MMFFOptimizeMolecule(protonated_mol)        # Optimizing Geometry

carbamate_mol = Chem.MolFromSmiles('CNC(=O)[O-]')   # Crating the other molecule in the same way
carbamate_mol = Chem.AddHs(carbamate_mol)
AllChem.EmbedMolecule(carbamate_mol)
AllChem.MMFFOptimizeMolecule(carbamate_mol)


#MA_mol2 = Chem.Mol(MA_mol)  # Create a copy

# Shifting carbamate molecule by +10 Angstroms along x-axis
conf = carbamate_mol.GetConformer()
for i in range(conf.GetNumAtoms()):
    pos = list(conf.GetAtomPosition(i))
    pos[0] += 10.0  # shift x-axis  so that molecules do not overlap
    conf.SetAtomPosition(i, pos)


combined = Chem.CombineMols(protonated_mol, carbamate_mol)   # Combining the molecules into One single system
#combined = Chem.CombineMols(combined, CO2_mol)              # For combining CO2
Chem.MolToMolFile(combined, "/Users/shibsankarmondal/PycharmProjects/CME502/merged_CN_CO2_state1.mol")  #Creating a molfile to visualize in Avogadro


In [ ]:
# Merged structure optimization
merged_state_mol = Chem.MolFromMolFile("./mol/merged_CN_CO2_state_organized.mol", sanitize=False)   # Opening the organized molecule

conf = merged_state_mol.GetConformer()
positions = conf.GetPositions()
symbols = [atom.GetSymbol() for atom in merged_state_mol.GetAtoms()]
ase_mol = Atoms(symbols=symbols, positions=positions)
n_total = len(ase_mol)
CO2_indices = list(range(n_total - 3, n_total))

constraint = FixInternals(bonds = [(0, 12)])
ase_mol.set_constraint(constraint)
merged_state_mol



conformer = merged_state_mol.GetConformer()
positions = conformer.GetPositions()
symbols = [atom.GetSymbol() for atom in merged_state_mol.GetAtoms()]

ase_mol = Atoms(symbols=symbols, positions=positions)
ase_mol.center(vacuum=5.0)

system = ase_mol
system.calc = XTB(method="GFN2-xTB", solvent='water')  # method can be GFN2-xTB or GFN1-xTB and fixing solvent
dyn = BFGS(ase_mol)        # Set optimizer
dyn.run(fmax=0.05)

# Saving the optimized structure
io.write('./mol/CH3NH2_State2_Opt.xyz', ase_mol, format='xyz')

In [ ]:
# Pathway calculation using NEB
os.environ["PATH"] = "/Users/shibsankarmondal/mamba/envs/cme502_2/bin:" + os.environ["PATH"]

print(shutil.which('dftb+'))     # Finds DFT bt in te system

# Initial and Final Structure
initial = io.read('./mol/CH3NH2_State1_Opt.xyz')
final = io.read('./mol/CH3NH2_State2_Opt.xyz')

# Adds 30 intermediate images
images = [initial]
images += [initial.copy() for _ in range(30)]
images += [final]

# Setting up the NEB object with climbing image and IDPP interpolation
neb = NEB(images, k=0.1, climb=True, allow_shared_calculator=False)
neb.interpolate(method='idpp')

#DFTB+ calculator to each image
for atoms in images:
    atoms.calc = Dftb(
        label='neb',
        atoms=atoms,
        Hamiltonian_SlaterKosterFiles_Prefix='./3ob-3-1/',
        Hamiltonian_MaxAngularMomentum_C='"p"',
        Hamiltonian_MaxAngularMomentum_H='"s"',
        Hamiltonian_MaxAngularMomentum_N='"p"',
        Hamiltonian_MaxAngularMomentum_O='"p"',
        Hamiltonian_MaxSCCIterations=10000,
        Hamiltonian_ReadInitialCharges='No',
        Hamiltonian_SCC='Yes',
        Hamiltonian_SCCTolerance=1e-5,
        Hamiltonian_Filling='Fermi { Temperature [Kelvin] = 1000 }',
        Hamiltonian_Solvation='''GeneralisedBorn { # GFN2-xTB/GBSA(water)
            Solvent = fromConstants {
                    Epsilon = 80.2
                    MolecularMass [amu] = 18.0
                    Density [kg/l] = 1.0
                }
                FreeEnergyShift [kcal/mol] = 1.16556316
                BornScale = 1.55243817
                BornOffset = 2.462811043694508E-02
                Radii = vanDerWaalsRadiiD3 {}
                Descreening = Values {
                    H = 0.71893869
                    C = 0.74298311
                    N = 0.90261230
                    O = 0.75369019
                }
                SASA {
                    ProbeRadius = 1.843075777670416
                    Radii = vanDerWaalsRadiiD3 {}
                    SurfaceTension = Values {
                    H = -3.34983060E-01
                    C = -7.47690650E-01
                    N = -2.31291292E+00
                    O = 9.17979110E-01
                }
            }
            HBondCorr = Yes
            HBondStrength = Values {
                H = -7.172800544988973E-02
                C = -2.548469535762511E-03
                N = -1.976849501504001E-02
                O = -8.462476828587280E-03
            }
        }
    '''
    )
#FIRE optimizer
opt = FIRE(neb, logfile='neb.log', dt=0.05, dtmax=0.2)
opt.run(fmax=4.5)

#saving NEB images
for i, img in enumerate(images):
    img.write(f'neb_image_{i:02d}.xyz')

    xyz_files = sorted(glob.glob("neb_image_*.xyz"))
images = [io.read(f) for f in xyz_files]

data = []

for img_idx, file in enumerate(xyz_files):
    atoms = io.read(file)
    try:
        energy = atoms.get_potential_energy()
    except:
        energy = None
    data.append({
        "image": img_idx,
        "energy (eV)": energy,
    })

In [ ]:
#Plotting the Reaction pathway vs Free Energy
df = pd.DataFrame(data)
plt.plot(df['image'], df["energy (eV)"], 'o-', color = 'plum', label = "Activation barrier curve")
plt.xlabel("Image number", fontweight='bold')
plt.ylabel("Energy (eV)", fontweight='bold')
plt.legend(fontsize='x-large')
plt.savefig("neb_com_energy.png", dpi=300)
plt.show()